In [167]:
import pandas as pd
df=pd.read_csv("floor0.csv")
fingerprint=df.iloc[:,:-9]
finger_location=df.iloc[:,-9:-7]

In [168]:
fingerprint

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
4,100,100,100,100,100,-90,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
4365,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
4366,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-63,-90,100,100,100
4367,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [169]:
finger_location

,LONGITUDE,LATITUDE
0,-7632.143600,4864982.217
1,-7474.553800,4864866.856
2,-7469.871500,4864864.365
3,-7466.445299,4864859.364
4,-7454.239100,4864829.266
...,...,...
4364,-7311.829500,4864805.727
4365,-7318.694800,4864795.302
4366,-7314.611700,4864800.712
4367,-7320.078900,4864794.328


In [170]:
import numpy as np
fingerprint=np.array(fingerprint)
finger_location=np.array(finger_location)
test_location=finger_location[-1]

In [171]:
class knn_location:
    def __init__(self,fingerprint,finger_location,test_location,test_finger):
        self.fingerprint=fingerprint
        self.finger_location=finger_location
        self.test_location=test_location
        self.test_finger=test_finger

    def compute(self):
        d_all=[]
        d_min=9999
        min_i=-1
        # 因为把最后一行提取出来作为待测位置
        for i in range(0,len(self.fingerprint)-1):
            # (np.sum((np.array([5,4])-np.array([1,1]))**2))**0.5 计算欧氏距离
            d=(np.sum((test_location-finger_location[i])**2))**0.5
            d_all.append(d)
            if d<d_min:
                d_min=d
                min_i=i
        print("d_min:",d_min,"num:",i,"\nlocation:",finger_location[i],"\nreal_location:",test_location)
        return d_all,d_min

In [164]:
knn=knn_location(fingerprint,finger_location,test_location,fingerprint[-1])
d_all,d_min=knn.compute()

d_min: 0.0 num: 4367 
location: [  -7320.0789 4864794.328 ] 
real_location: [  -7320.0789 4864794.328 ]


In [173]:
label=np.array(range(0,len(knn.fingerprint)-1))

In [174]:
def bubble(d_all,label,k): 
#  冒泡法 
  length = len(d_all) 
  for i in range(k): 
    for j in range(i+1,length): 
      if d_all[i] > d_all[j]: 
        d_all[j],d_all[i] = d_all[i],d_all[j]
        label[j],label[i]= label[i],label[j]
  return d_all[0:k],label[0:k]

In [181]:
k=10
d_sort,label_sort=bubble(d_all,label,k)
print(d_sort,"\n",label_sort)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
 [570 596 632 662 695 724 754 783 813 841]


In [184]:
k_location=finger_location[label_sort]
k_location

array([[  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ],
       [  -7320.0789, 4864794.328 ]])

In [198]:
np.sum(k_location.T,axis=1)/10

array([  -7320.0789, 4864794.328 ])